# Four point functions in the Potts and $O(n)$ models

This notebook reproduces some known results about Potts and $O(n)$ four-point functions, using the new Julia code.

In [1]:
Pkg.activate(".") # activate the parent environment
using BootstrapVirasoro
if Threads.nthreads() == 1
    println("You are using a single thread. Consider starting julia with more threads, for instance by setting
    the environment variable `export JULIA_NUM_THREADS=auto`")
end

  Activating project at `~/Documents/Recherche/projet_these/code/BootstrapVirasoro/examples`
[ Info: Precompiling BootstrapVirasoro [ca6c609c-7c6b-4ea6-be06-ee6e3ced7d6c] (cache misses: include_dependency fsize change (6), mismatched flags (12))


## Spectra

We create functions to generate the spectra of the $O(n)$ and Potts CFTs, with a cutoff:

\begin{align}
\mathcal{S}^{O(n)} &= \left\{V^d_{\langle 1,s\rangle}\right\}_{s\in 2\mathbb{N}+1} \bigcup \left\{V_{(r,s)}\right\}_{\substack{r\in \frac12\mathbb{N}^*\\ s\in\frac{1}{r}\mathbb{Z}}}  \ ,
 \\
 \mathcal{S}^{PSU(n)} &= \left\{V^d_{\langle 1,s\rangle}\right\}_{s\in\mathbb{N}^*} \bigcup \left\{V_{(r,s)}\right\}_{\substack{r\in \mathbb{N}^*\\ s\in\frac{1}{r}\mathbb{Z}}}  \ ,
 \\
 \mathcal{S}^\text{Potts} &= \left\{V^d_{\langle 1,s\rangle}\right\}_{s\in\mathbb{N}^*} \bigcup \left\{V_{(r,s)}\right\}_{\substack{r\in \mathbb{N}+2\\ s\in\frac{1}{r}\mathbb{Z}}} \bigcup  \left\{ V_{P_{(0,s)}}\right\}_{s\in \mathbb{N}+\frac12}\ .
\end{align}

## Bootstrap equations

We then numerically solve

\begin{align}
\sum_{V \in \mathcal{S}^{(s)}} D^{(s)}_V \mathcal I^{(s)}_V (x) = \sum_{V \in \mathcal{S}^{(t)}} D^{(t)}_V \mathcal I^{(t)}_V (x) = \sum_{V \in \mathcal{S}^{(u)}} D^{(u)}_V \mathcal I^{(u)}_V (x),
\end{align}

for some channel spectra $\mathcal{S}^{(s)}, \mathcal{S}^{(t)}, \mathcal{S}^{(u)}$, where $\mathcal I$ are interchiral conformal blocks, and the $D$'s are four-point structure constants.

We solve this system as
\begin{equation}
\underbrace{
\begin{bmatrix}
[\mathcal I^{(s)}_{V_j}(x_i)]_{ij} & [-\mathcal I^{(t)}_{V_j}(x_i)]_{ij} & [0] \\
[\mathcal I^{(s)}_{V_j}(x_i)]_{ij} & [0] & [-\mathcal I^{(u)}_{V_j}(x_i)]_{ij}
\end{bmatrix}}_A
\begin{bmatrix}
[D^{(s)}_{V_j}]_j \\
[D^{(t)}_{V_j}]_j \\
[D^{(u)}_{V_j}]_j
\end{bmatrix} = 
\begin{bmatrix}
\sum_{V_j \in \text{ known}} D^{(t)}_{V_j} \mathcal{I}^{(t)}_{V_j}(1-x_i) - \sum_{V_j \in \text{ known}} D^{(s)}_{V_j} \mathcal{I}^{(s)}_{V_j}(x_i)\\
\sum_{V_j \in \text{ known}} D^{(u)}_{V_j} \mathcal{I}^{(u)}_{V_j}(1/x_i) - \sum_{V_j \in \text{ known}} D^{(s)}_{V_j} \mathcal{I}^{(s)}_{V_j}(x_i)
\end{bmatrix}
\end{equation}
where the $x_i$ take more values than there are unknowns, i.e. $A$ is a tall rectangular matrix. 

To check numerical convergence, we solve two subsystems and compare the solutions. If the solutions are close, we know the computation has converged.


In [2]:
setprecision(BigFloat, 17, base=10)
c = CC(β=1/(big"0.8" + big"0.1"*im))
Δmax = 20.

function LoopFields(model)
    model === :On && return vcat(
        [Field(c, r=r, s=s) for r in 1:15 for s in -1+1//r:1//r:1],
        [Field(c, r=r, s=s) for r in 1//2:1:15 for s in -1+1//(2r):1//r:1 if (r*s)%1 == 0],
        [Field(c, r=1, s=s, diagonal=true) for s in 1:2:15]
    )
    model === :PSUn && return vcat(
        [Field(c, r=r, s=s) for r in 1:15 for s in -1+1//r:1//r:1],
        [Field(c, r=1, s=s, diagonal=true) for s in 1:15]
    )
    model === :Potts && return vcat(
        [Field(c, r=r, s=s) for r in 2:15 for s in -1+1//r:1//r:1],
        [Field(c, r=0, s=s, diagonal=true) for s in 1//2:1:3//2],
        [Field(c, r=1, s=s, diagonal=true) for s in 1:15]
    )
end

LoopSpectrum(model, Δmax) = Spectrum(LoopFields(model), Δmax, interchiral=true);

function solve_bootstrap(indices, Δmax; signature = Dict(:s => 0, :t => 0, :u => 0), diags = Dict())
    fields = [Field(c, r=r, s=s) for (r, s) in indices]
    co = Correlation(fields..., Δmax=Δmax)
    SOn = LoopSpectrum(:On, Δmax)

    Schan_On = ChannelSpectra(
        co, SOn,
        signature=signature
    )

    for (chan, VP) in diags
        add!(Schan_On, VP)
    end

    b = BootstrapSystem(Schan_On)
    compute_linear_system!(b)
    solve!(b)
    return b
end

solve_bootstrap (generic function with 1 method)

## $\langle (\frac{1}{2}, 0)^2 (1, 0)^2 \rangle$

We expect a three-dimensional space of solutions. We can distinguish between them by choosing an appropriate signature.

In [ ]:
using Random
Random.seed!(1234)
indices = ((1 // 2, 0), (1 // 2, 0), (1, 0), (1, 0))
system1 = solve_bootstrap(indices, Δmax, signature=Dict(:s => 1, :t => 1//2, :u => 3//2))
# system2 = solve_bootstrap(indices, Δmax, signature=Dict(:s => 0, :t => 3, :u => 3//2))
# system3 = solve_bootstrap(indices, Δmax, signature=Dict(:s => 1, :t => 3//2, :u => 1//2));

[ Info: system size: (84, 78)


BootstrapSystem{Complex{BigFloat}, ChannelSpectrum{Complex{BigFloat}}}(Complex{BigFloat}[0.307076848898485244 + 0.4517999345838628611im, 0.4709269143262821977 + 0.4473457588438643495im, 0.124349360512472723 + 0.4371753040058340423im, 0.4796482341908853142 + 0.3182570563149401122im, 0.1509951421467625454 + 0.1716891405979956153im, 0.2091628179571313195 + 0.3953763094576146497im, 0.235226297075436741 + 0.3306122380530927551im, 0.2882142672267687389 + 0.2150695468427042401im, 0.3502984736553532708 + 0.1807410817532131286im, 0.1212940999330415909 + 0.3169109558064803212im  …  0.4438615080258796475 + 0.2160810264890321852im, 0.4542137094551640386 + 0.4085694422101932011im, 0.1009617190019705818 + 0.4965870076440087044im, 0.2634898921342350153 + 0.4156207544726117087im, 0.3103579226000682256 + 0.3231026684614607403im, 0.1771876418628030858 + 0.1397444877114986106im, 0.3751476221230677366 + 0.4524183733340981783im, 0.3058655287552904944 + 0.1401444760544988122im, 0.4110931216256576981 + 0.272

In [10]:
system1.spectra

(s = channel s, Δmax = 20.0 + 0.0im
Non-diagonal:
(1, 0), (1, 1)
(2, -1//2), (2, 0), (2, 1//2), (2, 1)
(3, -2//3), (3, -1//3), (3, 0), (3, 1//3), (3, 2//3), (3, 1)
(4, -3//4), (4, -1//2), (4, -1//4), (4, 0), (4, 1//4), (4, 1//2), (4, 3//4), (4, 1)
(5, -4//5), (5, -3//5), (5, -2//5), (5, -1//5), (5, 0), (5, 1//5), (5, 2//5), (5, 3//5), (5, 4//5), (5, 1), t = channel t, Δmax = 20.0 + 0.0im
Non-diagonal:
(1//2, 0)
(3//2, -2//3), (3//2, 0), (3//2, 2//3)
(5//2, -4//5), (5//2, -2//5), (5//2, 0), (5//2, 2//5), (5//2, 4//5)
(7//2, -6//7), (7//2, -4//7), (7//2, -2//7), (7//2, 0), (7//2, 2//7), (7//2, 4//7), (7//2, 6//7)
(9//2, -8//9), (9//2, -2//3), (9//2, -4//9), (9//2, -2//9), (9//2, 0), (9//2, 2//9), (9//2, 4//9), (9//2, 2//3), (9//2, 8//9), u = channel u, Δmax = 20.0 + 0.0im
Non-diagonal:
(3//2, -2//3), (3//2, 0), (3//2, 2//3)
(5//2, -4//5), (5//2, -2//5), (5//2, 0), (5//2, 2//5), (5//2, 4//5)
(7//2, -6//7), (7//2, -4//7), (7//2, -2//7), (7//2, 0), (7//2, 2//7), (7//2, 4//7), (7//2, 6//7)
(

In [11]:
println(system1.consts)
# println(system2.consts)
# println(system3.consts)

Channel s


Fields         | Structure constants          | Relative errors            
--------------------------------------------------------------------
V_{(1, 0)}     |  1.00000e+00 + 0.00000e+00im | 0.00000e+00 + 0.00000e+00im
V_{(1, 1)}     | -6.39551e-07 - 1.91757e-06im |  1.82999e-03 + 0.00000e+00im
V_{(2, -1//2)} | -7.14618e-03 + 2.67175e-03im |  2.46519e+00 + 0.00000e+00im
V_{(2, 0)}     |  2.87929e-03 + 1.16625e-02im |  2.46380e+00 + 0.00000e+00im
V_{(2, 1//2)}  | -1.08285e-06 - 2.18959e-06im |  2.33724e+00 + 0.00000e+00im
V_{(2, 1)}     |  3.80903e-13 - 9.45838e-14im |  2.52635e+00 + 0.00000e+00im
V_{(3, -2//3)} |  1.27533e-06 - 1.54751e-06im |  2.53222e+00 + 0.00000e+00im
V_{(3, -1//3)} |  5.02183e-01 - 8.30317e-02im |  2.72178e-12 + 0.00000e+00im
V_{(3, 0)}     | -5.15222e-12 + 7.22931e-12im |  2.43758e+00 + 0.00000e+00im
V_{(3, 1//3)}  | -7.06648e-07 - 9.47806e-07im |  3.83581e-03 + 0.00000e+00im
V_{(3, 2//3)}  |  6.36532e-03 + 7.88828e-03im |  2.46565e+00 + 0.00000e+00im
V_{(3, 1)

In [ ]:
c = CC(β=1/big"0.8"+big"0.1"*im)
Δmax = 15
setprecision(BigFloat, 13)
indices = ((1 // 2, 0), (1 // 2, 0), (1, 0), (1, 0))
fields = [Field(c, r=r, s=s) for (r, s) in indices]
co = Correlation(fields..., Δmax=Δmax)
SOn = LoopSpectrum(:On, Δmax)

signatures = (s=1, t=1//2, u=3//2)
Schan_On = ChannelSpectra(co, SOn, signature=)